Banco Central do Brasil

PyPI version Build Status codecov
Sobre

    Obtenção de informações sobre taxas oficiais diárias de Inflação, Selic, Poupança e Dólar direto do site do Banco Central do Brasil (http://www.bcb.gov.br)

Pré requisitos

    Instalação de qualquer versão do Python (http://www.python.org/download)

Instalação das dependências

$ pip install -r feedparser

In [1]:
import feedparser
import re

USER_AGENT = "Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko"

class Inflacao:

	def __init__(self):
		self.inflacao_url = "http://conteudo.bcb.gov.br/api/feed/pt-br/PAINEL_INDICADORES/inflacao"
		self.feed = feedparser.parse(self.inflacao_url, agent=USER_AGENT)

	def get_meta_tax(self):
		feed = self.feed
		if feed['status'] == 200:
			for item in feed.entries:
				inflacao = item['summary']
				tax = re.search('<div id=label>Meta</div><div id=rate><div id=value>(\d+,\d+)</div>', inflacao).group(1)
			return tax
		else:
			return ("URL error %s" % self.inflacao_url)

	def get_acumulada_tax(self):
		feed = self.feed
		if feed['status'] == 200:
			for item in feed.entries:
				inflacao = item['summary']
				tax = re.search('<div id=label>Acumulada</div><div id=rate><div id=value>(\d+,\d+)</div>', inflacao).group(1)
			return tax
		else:
			return ("Site error %s" % self.inflacao_url)

class Poupanca:

	def __init__(self):
		self.poupanca_url = "http://conteudo.bcb.gov.br/api/feed/pt-br/PAINEL_INDICADORES/poupanca"
		self.feed = feedparser.parse(self.poupanca_url, agent=USER_AGENT)

	def get_poupanca_tax(self):
		feed = self.feed
		if feed['status'] == 200:
			for item in feed.entries:
				poupanca = item['summary']
				tax = re.search('<div id=value>(\d+,\d+)</div>', poupanca).group(1)
			return tax
		else:
			return ("URL error %s" % self.poupanca_url)

class Dolar:

	def __init__(self):
		self.dolar_url = "http://conteudo.bcb.gov.br/api/feed/pt-br/PAINEL_INDICADORES/cambio"
		self.feed = feedparser.parse(self.dolar_url, agent=USER_AGENT)

	def get_dolar_compra(self):
		feed = self.feed
		if feed['status'] == 200:
			for item in feed.entries:
				title = item['title']
				if re.search('(PTAX)', title) or (re.search('Euro', title)):
					continue
				tax = item['summary']
				compra = re.search('<div id=rate><div id=label>Compra</div><div id=value>(\d+,\d+)</div>', tax).group(1)

			return compra

		else:
			return ("URL error %s" % self.dolar_url)

	def get_dolar_venda(self):
		feed = self.feed
		if feed['status'] == 200:
			for item in feed.entries:
				title = item['title']
				if re.search('(PTAX)', title) or (re.search('Euro', title)):
					continue
				tax = item['summary']
				venda = re.search('<div id=rate><div id=label>Venda</div><div id=value>(\d+,\d+)</div>', tax).group(1)
			return venda
		else:
			return ("Site error %s" % self.dolar_url)

class Selic:

	def __init__(self):
		self.selic_url = "http://conteudo.bcb.gov.br/api/feed/pt-br/PAINEL_INDICADORES/juros"
		self.feed = feedparser.parse(self.selic_url, agent=USER_AGENT)


	def get_selic_real(self):
		feed = self.feed
		if feed['status'] == 200:
			for item in feed.entries:
				selic = item['summary']
				selic_real = re.search('<div id=dailyrate><div id=dailyratevalue>(\d+,\d+)</div>', selic).group(1)
			return selic_real
		else:
			return ("Site error %s" % self.selic_url)

In [2]:
#from bancocentral import Inflacao, Poupanca, Dolar, Selic

infl = Inflacao()
print(u'Inflação Meta: %s' % infl.get_meta_tax())
print(u'Inflação acumulada 12 últimos meses: %s' % infl.get_acumulada_tax())

poup = Poupanca()
print(u'Poupança: %s' % poup.get_poupanca_tax())

dolar = Dolar()
print(u'Dólar compra: %s' % dolar.get_dolar_compra())
print(u'Dólar venda: %s' % dolar.get_dolar_venda())

selic = Selic()
print(u'Selic diaria: %s' % selic.get_selic_real())


Inflação Meta: 4,5
Inflação acumulada 12 últimos meses: 2,8
Poupança: 0,3994
Dólar compra: 3,3203
Dólar venda: 3,3209
Selic diaria: 6,9
